In [ ]:
%pip install requests-cache

In [ ]:
import requests_cache
import yfinance as yf
import pandas as pd
import tensorflow as tf
import requests
import os
import datetime

from lts import LTSCell

In [ ]:
#Get data
df=yf.download("GOOGL", period="1y", interval="1d")
data = pd.DataFrame(df)
data.head()

In [ ]:
#Remove unnecessary columns
data.drop(columns=[
    'Adj Close'],
    inplace=True)

data.head()

In [ ]:
data.info()

In [ ]:
timesteps = data.index.to_numpy()
financials = data.to_numpy()

In [ ]:
timesteps, financials

In [ ]:
# predict 1 step at a time
HORIZON = 1

# use a week worth of timesteps to predict the horizon
WINDOW_SIZE = 7

In [ ]:
# Create sequential splits at a specified point (80% train, 20% test)
split_size = int(.8 * len(financials))

x_train, y_train = timesteps[:split_size], financials[:split_size]
x_test, y_test = timesteps[split_size:], financials[split_size:]

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
model_lts = tf.keras.Sequential([
    tf.keras.layers.RNN(LTSCell(32), time_major=True, return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu'),
    tf.keras.layers.Dense(HORIZON, activation='linear')
], name='model_lts')

In [ ]:
model_lts.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = .01),
    loss = tf.keras.losses.MAE,
    metrics = ['mae', 'mse']
)

In [ ]:
history = model_lts.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=500,
    callbacks=[
        create_model_checkpoint(model_name=model_lts.name),
        create_tensorboard_callback(
            dir_name='tensorboard_logs/tensorboard_logs_model_lts',
            experiment_name='model_lts'
        ),
        early_stopping,
        reduce_lr_plateau,
        
    ]
)